# Scrapper para obtener información de búsqueda de google 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
import subprocess
import random
import requests
import socket
import csv

## Conexión al navegador

Este espacio se utilza de prueba para la extracción de etiquetas en el s

In [28]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://google.com'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

## Lectura de archivos csv para obtener nombres

In [2]:
init=200
final=1000

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: '"'+X.replace('/', ' ').replace('*', '').rstrip()+'"')
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered).iloc[init:final]
df_input.head()

,Nombre
201,"""JUAN ANTONIO VEGA RODRIGUEZ"""
202,"""FRANCISCO JAVIER RODRIGUEZ RUIZ"""
203,"""CRISTIAN ADRIAN TOVAR TORRES"""
204,"""SANTIAGO EUSEBIO GONZALEZ"""
205,"""MA ISABEL BALDERAS RAMIREZ"""


# Scrapper

## Concatenado de etiquetas

In [8]:

data_ = []
data = []
nombres = []

start_url = 'https://google.com'
# Cargar la lista de proxies desde el archivo
with open ('proxies.csv', 'r') as file :
    proxies_list = file.read().splitlines()
    


for i in tqdm(df_input['Nombre']):
    
    # Opciones del controlador para google.com
    chrome_options = Options()
    
    # Elegir una dirección de proxy aleatoria de la lista para cada solicitud
    proxy = {'http': f'http://{random.choice(proxies_list)}'}
    print(proxy)
    # Configurar la solicitud con el proxy seleccionado
    response = requests.get(start_url, proxies=proxy)
    
    time.sleep(2)

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(start_url)
    
    # Acción de búsqueda en el input 
    nombres.append(i)
    wait = WebDriverWait(driver, 100000)
    nombre = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'gLFyf')))
    nombre.send_keys(i)
    nombre.send_keys(Keys.ENTER)

    # Esperar a que los resultados de búsqueda se carguen completamente
    time.sleep(2)

    # Obtener el HTML de la página actual
    html_link = driver.page_source
    soup = BeautifulSoup(html_link, 'html.parser')

    # Extracción de clases
    div = soup.find_all('div', class_="kvH3mc BToiNc UK95Uc")
    cont = soup.find_all('div', class_="Z26q7c UK95Uc")

    # Primer bucle para extraer la clase que conforma el bloque que corresponde al URL desplegado
    if div:
        for tag in div:
            href_tags = tag.find_all('a', href=True)
            text = tag.text

            for href_tag in href_tags:
                href = href_tag['href']

                data.append({'href': href, 'Text': text, 'Nombre': i})
    else:
        data.append({'href': None, 'Text': None, 'Nombre': i})

    # Segundo bucle para extraer los resúmenes de las URL desplegadas
    if cont:
        for tag in cont:
            text = ''

            span_tags = tag.find_all('span')
            if span_tags:
                for span_tag in span_tags:
                    if span_tag.get('class', []) == ['ITHCWe']:
                        continue
                    text += span_tag.text.strip() + ' '
            else:
                text = tag.text.strip()

            data_.append({'Text': text.strip()})
    else:
        data_.append({'Text': None})

    #google = driver.find_element(By.CLASS_NAME, 'logo Ib7Efc')
    #google.click()
    
    driver.quit()

    # Rellenar los datos faltantes en data y data_ para tener la misma longitud
    max_length = max(len(data), len(data_))
    data += [{'href': None, 'Text': None}] * (max_length - len(data))
    data_ += [{'Text': None}] * (max_length - len(data_))

df = pd.DataFrame(data)
df_2 = pd.DataFrame(data_)

df_3 = pd.concat([df, df_2], axis=1)
df_3.dropna(inplace=True)
# Filtrado por tipo de página encontrada
# df_3 = df_3[df_3['href'].str.contains('facebook|twitter|instagram|linkedin|misprofesores|.org|tiktok', case=False)]
# Se quitan enlaces para traducir páginas
df_3 = df_3[~df_3['href'].str.contains('translate.google')]


# Se guarda el df en un csv
df_3.to_csv('Z:/Data/HSBC/SCRAPPING/200_1000_.csv')



  0%|          | 0/800 [00:00<?, ?it/s]

{'http': 'http://http://162.223.94.164:80'}


  0%|          | 1/800 [00:10<2:22:36, 10.71s/it]

{'http': 'http://http://130.61.49.108:3128'}


  0%|          | 2/800 [00:20<2:17:23, 10.33s/it]

{'http': 'http://http://155.94.241.132:3128'}


  0%|          | 2/800 [00:23<2:35:05, 11.66s/it]


KeyboardInterrupt: 